# Лабораторная работа 1. Введение в машинное обучение. Обучение с учителем. Задача регрессии

<b>Традиционное предупреждение для всех лабораторных работ:</b> перед обучением моделей необходимо выполнить предварительную обработку данных, которая <b>обязательно</b> включает в себя:
- заполнение пропущенных значений (рекомедуется логика заполнения пропусков на основе типа данных, которая использовалась в РГР по Практикуму);
- преобразование категориальных признаков в числовые (используйте one-hot кодирование или map; используйте знания с Практикума).

Предобработка может включать в себя другие действия, но выполнение описанных выше действий обязательно.

Сделайте это один раз и сохраните в отдельный csv файл, а потом его используйте.

<b>Выполните следующие задания:</b>
- загрузите датасет для регрессии, выделите целевой признак и предикторы, разбейте данные на обучающую и тестовую выборку;
- решите задачу регрессии на ваших данных с использованием моделей sklearn (линейная регрессия + L1, L2), для моделей с регуляризациями подберите гиперпараметр;
- решите задачу регрессии на ваших данных с использованием моделей sklearn (полиномиальная регрессия + L1, L2), для моделей с регуляризациями подберите гиперпараметр;
- вычислите значения метрик $R^2$, MAE, MSE, RMSE, MAPE для всех обученных моделей; выберите лучшую модель;
- самостоятельно реализуйте (желательно в виде класса) модель линейной регрессии с регуляризацией (можете выбрать L1 или L2);
- самостоятельно реализуйте вычисление всех используемых метрик (в виде функций, принимающих два аргумента);
- обучите вашу модель линейной регрессии на ваших данных; оцените качество с помощью реализованных вами метрик.

## Предобработка

In [2]:
import pandas as pd
import numpy as np

dsredwine = pd.read_csv("../data/winequality-red.csv", delimiter =";").drop_duplicates()
dsredwine["type"] = list("red" for i in range(len(dsredwine)))

dsredwine.info()


FileNotFoundError: [Errno 2] No such file or directory: '../data/winequality-red.csv'

In [14]:
dswhitewine = pd.read_csv("../data/winequality-white.csv", delimiter =";").drop_duplicates()
dswhitewine["type"] = list("white" for i in range(len(dswhitewine)))

dswhitewine.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3961 entries, 0 to 4897
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         3961 non-null   float64
 1   volatile acidity      3961 non-null   float64
 2   citric acid           3961 non-null   float64
 3   residual sugar        3961 non-null   float64
 4   chlorides             3961 non-null   float64
 5   free sulfur dioxide   3961 non-null   float64
 6   total sulfur dioxide  3961 non-null   float64
 7   density               3961 non-null   float64
 8   pH                    3961 non-null   float64
 9   sulphates             3961 non-null   float64
 10  alcohol               3961 non-null   float64
 11  quality               3961 non-null   int64  
 12  type                  3961 non-null   object 
dtypes: float64(11), int64(1), object(1)
memory usage: 433.2+ KB


In [53]:
#слишком большой объем данных, выберем 1000
dswine = pd.merge(dswhitewine.iloc[0:500], dsredwine.iloc[0:500], how="outer")

dswine


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,white
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,white
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,white
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,white
4,6.2,0.32,0.16,7.0,0.045,30.0,136.0,0.9949,3.18,0.47,9.6,6,white
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,10.2,0.24,0.49,2.4,0.075,10.0,28.0,0.9978,3.14,0.61,10.4,5,red
996,10.5,0.59,0.49,2.1,0.070,14.0,47.0,0.9991,3.30,0.56,9.6,4,red
997,10.6,0.34,0.49,3.2,0.078,20.0,78.0,0.9992,3.19,0.70,10.0,6,red
998,12.3,0.27,0.49,3.1,0.079,28.0,46.0,0.9993,3.20,0.80,10.2,6,red


**One-hot encoding**
https://analyticsindiamag.com/a-complete-guide-to-categorical-data-encoding/

In [54]:
import category_encoders as ce

encoder=ce.OneHotEncoder(cols='type',handle_unknown='return_nan',return_df=True,use_cat_names=True)

dswine_encoded = encoder.fit_transform(dswine)

dswine_encoded.to_csv("../data/dswine_encoded.csv", index=False)

dswine_encoded


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type_white,type_red
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,1.0,0.0
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,1.0,0.0
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,1.0,0.0
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,1.0,0.0
4,6.2,0.32,0.16,7.0,0.045,30.0,136.0,0.9949,3.18,0.47,9.6,6,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,10.2,0.24,0.49,2.4,0.075,10.0,28.0,0.9978,3.14,0.61,10.4,5,0.0,1.0
996,10.5,0.59,0.49,2.1,0.070,14.0,47.0,0.9991,3.30,0.56,9.6,4,0.0,1.0
997,10.6,0.34,0.49,3.2,0.078,20.0,78.0,0.9992,3.19,0.70,10.0,6,0.0,1.0
998,12.3,0.27,0.49,3.1,0.079,28.0,46.0,0.9993,3.20,0.80,10.2,6,0.0,1.0


In [23]:
#START
import pandas as pd
from sklearn.model_selection import train_test_split

dfwine = pd.read_csv("../data/dswine_encoded.csv", delimiter =",")

y = dfwine["quality"]
X = dfwine.drop(["quality"], axis=1)

y


0      6
1      6
2      6
3      6
4      6
      ..
995    5
996    4
997    6
998    6
999    4
Name: quality, Length: 1000, dtype: int64

In [24]:
X

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type_white,type_red
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,1.0,0.0
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,1.0,0.0
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,1.0,0.0
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,1.0,0.0
4,6.2,0.32,0.16,7.0,0.045,30.0,136.0,0.9949,3.18,0.47,9.6,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,10.2,0.24,0.49,2.4,0.075,10.0,28.0,0.9978,3.14,0.61,10.4,0.0,1.0
996,10.5,0.59,0.49,2.1,0.070,14.0,47.0,0.9991,3.30,0.56,9.6,0.0,1.0
997,10.6,0.34,0.49,3.2,0.078,20.0,78.0,0.9992,3.19,0.70,10.0,0.0,1.0
998,12.3,0.27,0.49,3.1,0.079,28.0,46.0,0.9993,3.20,0.80,10.2,0.0,1.0


In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((900, 13), (100, 13), (900,), (100,))

In [26]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso

lr = LinearRegression().fit(X_train, y_train)

y_pred = lr.predict(X_test)

In [27]:
#Linear
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from math import sqrt

#средняя абсолютная ошибка (Mean Absolute Error, MAE)
#средняя квадратичная ошибка (Mean Squared Error, MSE)
#квадратный корень из средней квадратичной ошибки (Root Mean Squared Error)
#средняя абсолютная ошибка в процентах (Mean Absolute Percentage Error, MAPE)
#коэффициент детерминации R^2

print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')
print(f'R^2: {lr.score(X_test, y_test)}')
lr.coef_

MAE: 0.543663994899126
MSE: 0.474197042044042
RMSE: 0.688619664287945
MAPE: 0.0950994450337928
R^2: 0.28151963326660323


array([ 7.11563839e-02, -1.15413448e+00, -1.22269521e-01,  4.25737851e-03,
       -1.48123749e+00,  8.19065355e-03, -2.80430001e-03, -4.01480610e+01,
        5.55195857e-01,  8.08587822e-01,  2.61411507e-01,  7.53340189e-02,
       -7.53340189e-02])

In [28]:
#L2
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import numpy as np

parameters = {'alpha': np.arange(0, 1, 0.005)}
ridge_optimal = RandomizedSearchCV(Ridge(), parameters).fit(X_train, y_train)
display(ridge_optimal.best_params_)

ridge = Ridge(alpha = ridge_optimal.best_params_["alpha"]).fit(X_train, y_train)
y_pred = ridge.predict(X_test)

print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')
print(f'R^2: {ridge.score(X_test, y_test)}')
ridge.coef_

{'alpha': 0.27}

MAE: 0.5390761995132514
MSE: 0.46907781145231425
RMSE: 0.6848925546772386
MAPE: 0.09416760708120968
R^2: 0.2892760432540694


array([ 0.03765027, -1.17815839, -0.14024464, -0.01053374, -1.28836714,
        0.00805383, -0.00285126, -0.05463131,  0.40785017,  0.7250413 ,
        0.29986295,  0.12484001, -0.12484001])

In [29]:
#L1
lasso_optimal = RandomizedSearchCV(Lasso(), parameters).fit(X_train, y_train)
display(lasso_optimal.best_params_)

lasso = Lasso(alpha = lasso_optimal.best_params_["alpha"]).fit(X_train, y_train)
y_pred = lasso.predict(X_test)

print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')
print(f'R^2: {lasso.score(X_test, y_test)}')
lasso.coef_

{'alpha': 0.19}

MAE: 0.6272312356101957
MSE: 0.5729691761814142
RMSE: 0.7569472743734627
MAPE: 0.11039164768686577
R^2: 0.13186488457361512


array([ 0.00000000e+00, -0.00000000e+00,  0.00000000e+00, -1.11266820e-04,
       -0.00000000e+00,  6.46170746e-03, -1.20366479e-03, -0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  2.01528522e-01,  0.00000000e+00,
       -0.00000000e+00])

In [32]:
#polinom
from sklearn.preprocessing import PolynomialFeatures

p = PolynomialFeatures(2)  
X_train_p=p.fit_transform(X_train)
X_test_p=p.fit_transform(X_test)

lr_p=LinearRegression().fit(X_train_p, y_train)
y_pred=lr_p.predict(X_test_p)

print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')
print(f'R^2: {lr_p.score(X_test_p, y_test)}')
lr_p.coef_


MAE: 0.5728373997554445
MSE: 0.554128904673771
RMSE: 0.7443983508000075
MAPE: 0.1003051997174006
R^2: 0.16041075049428666


array([-8.61784868e-03,  8.46455835e+01,  1.25280250e+02, -1.49909354e+02,
        1.00853047e+01,  1.56792554e+02,  1.71305924e-01,  2.60477887e+00,
       -8.38400485e+04,  6.04832862e+02,  2.47081424e+02, -2.02975163e+01,
       -3.90039379e+01,  3.90039456e+01,  3.06092948e-02,  1.63623785e-02,
        5.65960970e-02,  2.60110556e-02, -1.55424984e+00,  5.17171476e-04,
        2.03538436e-03, -1.29791202e+02,  5.28157719e-01,  8.05180592e-01,
       -4.99576209e-02,  4.21909028e+01,  4.24549202e+01, -2.71395923e-01,
       -1.71277174e+00,  1.76402156e-01,  1.73262339e+00, -3.20060097e-03,
        1.50077230e-02, -2.00362788e+02,  8.84609126e-01,  4.85559318e-01,
        5.48478874e-01,  6.12813421e+01,  6.39989205e+01, -3.08328979e-01,
       -5.08144491e-02,  9.22983552e+00,  2.08563159e-02,  6.16705857e-03,
        2.20262170e+02,  3.62997223e-01, -1.95228636e+00,  4.30866066e-01,
       -7.46768841e+01, -7.52324770e+01, -2.95547971e-03, -3.34556795e-01,
        9.96034901e-04,  

In [33]:
#polinomL2
ridge_p_optimal = RandomizedSearchCV(Ridge(), parameters).fit(X_train_p, y_train)
display(ridge_p_optimal.best_params_)

ridge_p = Ridge(alpha = ridge_p_optimal.best_params_["alpha"]).fit(X_train_p, y_train)
y_pred = ridge_p.predict(X_test_p)

print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')
print(f'R^2: {ridge_p.score(X_test_p, y_test)}')
ridge_p.coef_


{'alpha': 0.6900000000000001}

MAE: 0.5655919217958228
MSE: 0.5225805842317671
RMSE: 0.7228973538696674
MAPE: 0.09856521843730395
R^2: 0.20821123601247427


array([ 0.00000000e+00, -1.07990747e-01, -2.75815863e-01, -7.10165563e-02,
        3.23691721e-02,  9.70101673e-02,  3.67751572e-02,  7.48861234e-01,
        3.07907037e-03,  1.49413695e-01, -8.04073079e-02, -3.96014309e-01,
       -3.19774629e-01,  3.19774629e-01, -1.45707176e-02, -3.00317915e-01,
        4.30816804e-02,  3.05878774e-03, -4.90280514e-02,  2.10357327e-03,
        2.37903910e-04, -6.46710245e-03,  6.63436415e-02,  3.88078430e-02,
        2.62814363e-02, -1.47987425e-01,  3.99966788e-02, -1.15228402e-02,
       -2.15288296e-01,  9.29560550e-02,  1.11297554e-01, -7.52390156e-03,
        7.03679661e-03, -2.72449071e-01, -8.56743083e-01,  2.87663946e-02,
        3.75812649e-01, -1.02201701e+00,  7.46201150e-01,  4.52182928e-02,
        2.12247644e-02,  9.05586622e-02,  1.11028711e-02,  2.40902558e-03,
       -6.44390961e-02, -1.67921070e-02, -1.94367360e-01, -5.54447989e-02,
        5.45056052e-02, -1.25522162e-01, -7.33293681e-04, -9.67520814e-02,
        9.53767768e-04,  

In [1]:
#polinomL1
parameter = {"alpha" : np.arange(0.1, 1, 0.05)}

lasso_p_optimal = GridSearchCV(Lasso(), parameter).fit(X_train_p, y_train)
display(lasso_p_optimal.best_params_)
#lasso_p_optimal.best_params_["alpha"]

lasso_p = Lasso(alpha = lasso_p_optimal.best_params_["alpha"]).fit(X_train_p, y_train)
y_pred = lasso_p.predict(X_test_p)

print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')
print(f'R^2: {lasso_p.score(X_test_p, y_test)}')
lasso_p.coef_

NameError: name 'np' is not defined

 $\hat{y}_i=\sum_{j=1}^{n}{{\omega_{j}}{X_{ij}}}$


L2: $L=\frac{1}{m}[y-(\sum_{j=1}^{n}{{w_{j}}{X_{ij}}})]^2+\frac{1}{m}\alpha{\sum_{j=1}^n{w_j^2}}$.

In [11]:
from math import sqrt

def mae(y_test, y_pred):
    return np.mean(np.abs(y_test - y_pred))

def mse(y_test, y_pred):
    return np.mean((y_test - y_pred)**2)

def rmse(y_test, y_pred):
    return sqrt(mse(y_test, y_pred))

def mape(y_test, y_pred):
    return np.mean(np.abs((y_test - y_pred)/y_test))

def r2(y_test, y_pred, x_train):
    return 1 - mse(y_test, y_pred)/(np.mean((y_test - np.mean(x_train, axis=1))**2))

In [36]:
class LassoRegression:

    def fit(self, X, Y):

        self.m, self.n = X.shape
        self.W = np.zeros(self.n)
        self.b = 0
        self.X = X
        self.Y = Y
        for i in range(2000):
            self.train()
        return self
    
    def train(self):
        Y_predict = self.predict(self.X)
        
        dW = (-(2 * (self.X.T).dot(self.Y - Y_predict)) + 0.2) / self.m
        db = -2 * np.sum(self.Y - Y_predict) / self.m
        
        self.W = self.W - 0.00001*dW
        self.b = self.b - 0.00001*db
        return self

    def predict(self, X):
        return X.dot(self.W)+self.b

In [37]:
model = LassoRegression()
model.fit(X_train, y_train)

y_pred_my = model.predict(X_test)

print(f'MAE: {mae(y_test, y_pred_my)}')
print(f'MSE: {mse(y_test, y_pred_my)}')
print(f'RMSE: {rmse(y_test, y_pred_my)}')
print(f'MAPE: {mape(y_test, y_pred_my)}')
print(f'R^2: {r2(y_test, y_pred_my, X_test)}')


MAE: 0.6967960375275275
MSE: 0.7768840569009129
RMSE: 0.8814102659380096
MAPE: 0.12073362231365203
R^2: 0.9862378538043515
